# C4.5 Algorithm Implement 

## ID3: information gain 
## C4.5: ratio of information gain 

In [1]:
import numpy as np


# for numeric data (need encoding data to number).
class C45:

    def __init__(self, maxDepth=3, minGain=0):
        self.__maxDepth = maxDepth
        self.__minGain = minGain
        self.__tree = []

    # given the y in that dataset , compute the entropy
    def _entropy(self, y):
        values, counts = np.unique(y, return_counts=True)
        total = len(y)
        entropy = -np.sum([count / total * np.log2(count / total) for count in counts])
        return entropy

    # given attribute index and that dataset(x,y), compute conditional entropy
    def _conditEntropy(self, attribute, x, y):
        values = np.unique(x[:, attribute])
        entropy = []
        # iterate each value in that attribute
        for value in values:
            index = np.nonzero(value == x[:, attribute])
            # how many data points belongs to that attribute value
            label = y[index]
            weight = len(label) / len(y)
            h = self._entropy(label)
            attrEntrop = weight * h
            entropy.append(attrEntrop)
        conditEntropy = np.sum(entropy)
        return conditEntropy

    def C45Train(self, x, y, depth=0, features=None):
        if np.any(features) == None:
            features = np.arange(x.shape[1])
        # if no data return the tree
        if len(y) == 0:
            return self.__tree
        # if all data belong to one class, then return
        values, counts = np.unique(y, return_counts=True)
        # if only a class, return the tree
        if len(values) == 1:
            # depth and its class
            if depth == 0:
                #(tree depth, feature, value, class)
                self.__tree.append((depth, 'Root Node', None, values.item()))
            return self.__tree
        # if no features, return, # if it is root, it is impossible the length of feature is 0
        elif len(features) == 0 or depth >= self.__maxDepth:
            return self.__tree
        else:
            # start building decision tree
            if depth == 0:
                valMaxCount = values[np.argmax(counts)]
                self.__tree.append((depth, 'Root Node', None, valMaxCount))
            # 1. compute data entropy
            h = self._entropy(y)
            # 2. compute conditional entropy of each feature
            conEL = []
            for feature in features:
                conditEntropy = self._conditEntropy(feature, x, y)
                conEL.append(conditEntropy)
            # 3. compute gain
            gain = h - np.array(conEL)
            # add this part and change the criterion variable with infGain
            #==============* c4.5=============
            infGain=[]
            for i,feature in enumerate(features):
                hf = self._entropy(x[:, feature])
                # if hf=0 => H(D|A)=H(D) => gain=0 => infGain=0
                if hf==0:
                    infGainE=0
                else:
                    infGainE=gain[i]/hf
                infGain.append(infGainE)
            #==============* c4.5=============
            # 4. max gain
            maxGain = np.max(infGain)  # changed
            # if max gain less than threshold
            if maxGain <= self.__minGain:
                return self.__tree
            # else split the to sub-nodes
            else:
                # 5. get feature with max gain and set it as standard to split data
                maxGainF = features[np.argmax(infGain)]  # changed
                # 6. according to the feature values to split dataset
                attrValues = np.unique(x[:, maxGainF])
                depth += 1
                tree = None
                for attrValue in attrValues:
                    xAttrIndex = np.nonzero(attrValue == x[:, maxGainF])
                    xsub = x[xAttrIndex]
                    ysub = y[xAttrIndex]
                    subvals, subcts = np.unique(ysub, return_counts=True)
                    val = subvals[np.argmax(subcts)]
                    self.__tree.append((depth, maxGainF, attrValue, val))
                    leftFeatures = features[np.nonzero(maxGainF != features)]
                    tree = self.C45Train(xsub, ysub, depth,leftFeatures)
                # tree = self.__tree
                return tree

    # Node structure: (tree depth,feature,value,class)
    def predict(self, x):
        predL = []
        # for each data points
        for data in x:
            # a data point has match history in the tree
            matchL = []
            for attribute, value in enumerate(data):
                # iterative tree
                for node in self.__tree:
                    # if match any tree node, add that node to match history
                    if attribute == node[1]:
                        if value == node[2]:
                            matchL.append((node[0], node[3]))
                        else:
                            continue
                    else:
                        continue
            # find class with max tree depth as the final prediction
            pred = max(matchL, key=lambda ele: ele[0])[1]
            predL.append(pred)
        return predL

    def accuracy(self, x, y):
        predL = self.predict(x)
        return np.mean(y == predL)

In [2]:
# generate history data
np.random.seed(42)
hx=np.random.randint(10,size=(100,5))
hy=np.random.choice(np.arange(3),size=100,replace=True)

# generate test data
np.random.seed(0)
tx=np.random.randint(10,size=(10,5))
ty=np.random.choice(np.arange(3),size=10,replace=True)

In [3]:
dt=C45()
tree=dt.C45Train(hx,hy)
predicts=dt.predict(tx)
accuracy=dt.accuracy(tx,ty)
print(f'tree={tree}\npredicts={predicts}\naccuray={accuracy}')

tree=[(0, 'Root Node', None, 0), (1, 2, 0, 0), (2, 0, 1, 0), (2, 0, 4, 1), (2, 0, 6, 0), (3, 1, 1, 0), (3, 1, 4, 1), (2, 0, 7, 2), (2, 0, 8, 0), (1, 2, 1, 0), (2, 1, 0, 2), (2, 1, 3, 1), (2, 1, 4, 0), (2, 1, 5, 0), (2, 1, 6, 0), (2, 1, 9, 0), (1, 2, 2, 0), (2, 1, 0, 0), (2, 1, 1, 1), (2, 1, 3, 2), (2, 1, 5, 0), (2, 1, 6, 0), (2, 1, 7, 2), (2, 1, 8, 0), (2, 1, 9, 0), (1, 2, 3, 0), (2, 1, 0, 0), (3, 0, 2, 0), (3, 0, 3, 2), (2, 1, 1, 2), (2, 1, 2, 1), (2, 1, 4, 0), (2, 1, 5, 0), (2, 1, 8, 0), (2, 1, 9, 1), (1, 2, 4, 2), (2, 3, 0, 1), (3, 0, 2, 1), (3, 0, 5, 1), (3, 0, 7, 0), (2, 3, 2, 0), (3, 0, 0, 0), (3, 0, 8, 1), (2, 3, 3, 2), (2, 3, 4, 0), (2, 3, 5, 0), (2, 3, 6, 2), (3, 0, 0, 1), (3, 0, 3, 2), (2, 3, 8, 2), (2, 3, 9, 2), (1, 2, 5, 1), (2, 1, 1, 1), (3, 0, 0, 1), (3, 0, 1, 1), (3, 0, 3, 0), (2, 1, 3, 0), (2, 1, 4, 2), (2, 1, 5, 1), (2, 1, 6, 2), (2, 1, 9, 2), (1, 2, 6, 1), (2, 1, 0, 2), (2, 1, 2, 1), (2, 1, 3, 0), (2, 1, 6, 0), (2, 1, 7, 2), (2, 1, 8, 0), (3, 0, 4, 0), (3, 0, 9, 1), (